# 编程和测试要求
## 5. 工作模式

> 基于S-AES算法，使用密码分组链(CBC)模式对较长的明文消息进行加密。注意初始向量(16 bits) 的生成，并需要加解密双方共享。
> 
> 在CBC模式下进行加密，并尝试对密文分组进行替换或修改，然后进行解密，请对比篡改密文前后的解密结果。

In [136]:
# 加载实现的加密算法类
from SAES.SAES import SAES

### 5.1 CBC工作模式的实现

在本小节中，将选取测试用例为:

In [137]:
# CBC工作模式的测试用例
plaintext = "Hello S-AES and CBC!"
key = "1101001110100101"
iv = "0101101000001111"

#### 5.1.1 采用CBC模式的加密过程

In [138]:
def cbc_encrypt(plaintext, key, iv):
    saes = SAES(key=key)
    previous_cipherblock = iv
    ciphertext = ""

    # 转换输入为二进制
    binary_input = ''.join([bin(ord(char)).replace("0b", "").zfill(8) for char in plaintext])

    # 检查字符数并进行填充
    if len(plaintext) % 2 == 1:  # 奇数字符，添加00000001
        binary_input += '00000001'
    else:  # 偶数字符，添加00000010 00000010
        binary_input += '00000010' + '00000010'

    # 使用CBC模式进行加密
    for i in range(0, len(binary_input), 16):
        block = binary_input[i:i + 16]

        # 明文块与前一个密文块异或
        xor_block = ''.join([str(int(a) ^ int(b)) for a, b in zip(block, previous_cipherblock)])

        cipherblock = saes.encrypt(xor_block)
        ciphertext += cipherblock

        previous_cipherblock = cipherblock

    # 将结果转换为十六进制字符串
    hex_ciphertext = hex(int(ciphertext, 2))[2:].zfill(len(ciphertext) // 4)

    return hex_ciphertext


cbc_encrypt函数接收用户字符输入plaintext,约定的key以及iv初始向量。

In [139]:
ciphertext = cbc_encrypt(plaintext, key, iv)
print(f'密文是:{ciphertext}')

密文是:a95ea41bda4db7b2672fc73a5af31b90e6444fc43b63


#### 5.1.2 对CBC模式的解密过程

In [140]:
def cbc_decrypt(ciphertext, key, iv):
    saes = SAES(key=key)
    previous_cipherblock = iv
    binary_output = ""

    # 从十六进制字符串转换为二进制
    binary_input = bin(int(ciphertext, 16))[2:].zfill(len(ciphertext) * 4)

    # 使用CBC模式进行解密
    for i in range(0, len(binary_input), 16):
        block = binary_input[i:i + 16]

        decrypted_block = saes.decrypt(block)

        # 解密块与前一个密文块异或
        xor_block = ''.join([str(int(a) ^ int(b)) for a, b in zip(decrypted_block, previous_cipherblock)])
        binary_output += xor_block

        previous_cipherblock = block

    # 移除填充
    if binary_output[-8:] == '00000001':
        binary_output = binary_output[:-8]
    elif binary_output[-16:] == '00000010' + '00000010':
        binary_output = binary_output[:-16]

    # 将二进制转换回ASCII
    ascii_output = ''.join([chr(int(binary_output[i:i + 8], 2)) for i in range(0, len(binary_output), 8)])

    return ascii_output

cbc_decrypt函数接收16进制字符串,约定的key以及iv初始向量。

In [141]:
decrypted_text = cbc_decrypt(ciphertext, key, iv)
print(f'解密后的明文是:{decrypted_text}')

解密后的明文是:Hello S-AES and CBC!


测试用例通过,CBC密码分组链实现结束

#### 5.2 篡改密文组结果分析

##### 5.2.1 在加密过程中被篡改:
篡改第一个密文组末位,定义函数用以模拟此情况。

In [142]:
def cbc_encrypt_tamper(plaintext, key, iv, tamper_first_block=False):
    saes = SAES(key=key)
    previous_cipherblock = iv
    ciphertext = ""
    blocks = []  # 用于存储每个加密块，以便后续输出和比较

    # 转换输入为二进制
    binary_input = ''.join([bin(ord(char)).replace("0b", "").zfill(8) for char in plaintext])

    # 检查字符数并进行填充
    if len(plaintext) % 2 == 1:  # 奇数字符，添加00000001
        binary_input += '00000001'
    else:  # 偶数字符，添加00000010 00000010
        binary_input += '00000010' + '00000010'

    # 使用CBC模式进行加密
    for i in range(0, len(binary_input), 16):
        block = binary_input[i:i + 16]

        # 明文块与前一个密文块异或
        xor_block = ''.join([str(int(a) ^ int(b)) for a, b in zip(block, previous_cipherblock)])

        cipherblock = saes.encrypt(xor_block)

        # 篡改第一个块的最后一个bit
        if i == 0 and tamper_first_block:
            cipherblock = cipherblock[:15] + ('1' if cipherblock[15] == '0' else '0') + cipherblock[16:]
      
        ciphertext += cipherblock
        blocks.append(cipherblock)
        previous_cipherblock = cipherblock
    # 将结果转换为十六进制字符串
    hex_ciphertext = hex(int(ciphertext, 2))[2:].zfill(len(ciphertext) // 4)

    return hex_ciphertext


In [143]:
ciphertext1 = cbc_encrypt_tamper(plaintext,key,iv)
plaintext1 = cbc_decrypt(ciphertext1,key,iv)

ciphertext2 = cbc_encrypt_tamper(plaintext,key,iv,True)
plaintext2 = cbc_decrypt(ciphertext2,key,iv)

print("未被篡改的密文:",ciphertext1)
print("被篡改后的密文:",ciphertext2)
# 
print("未被篡改的解密明文:",plaintext1)
print("被篡改后的解密明文:",plaintext2)

未被篡改的密文: a95ea41bda4db7b2672fc73a5af31b90e6444fc43b63
被篡改后的密文: a95f741c6a4727b47720a7348aff8b98864f3fc11b60
未被篡改的解密明文: Hello S-AES and CBC!
被篡改后的解密明文: (kllo S-AES and CBC!


发现篡改的效果并不明显,只有在更改的block才有影响。

所以得出结论：在没有其他安全措施情况下,在加密过程中被篡改是不安全的,加密算法需要保证连续性

#### 5.2.2 加密完成后密文分组被篡改:

依然是篡改密文分组block1的末位

In [144]:
def cbc_encrypt_tamper(plaintext, key, iv, tamper_first_block=False):
    saes = SAES(key=key)
    previous_cipherblock = iv
    ciphertext = ""
    blocks = []  # 用于存储每个加密块，以便后续输出和比较

    # 转换输入为二进制
    binary_input = ''.join([bin(ord(char)).replace("0b", "").zfill(8) for char in plaintext])

    # 检查字符数并进行填充
    if len(plaintext) % 2 == 1:  # 奇数字符，添加00000001
        binary_input += '00000001'
    else:  # 偶数字符，添加00000010 00000010
        binary_input += '00000010' + '00000010'

    # 使用CBC模式进行加密
    for i in range(0, len(binary_input), 16):
        block = binary_input[i:i + 16]

        # 明文块与前一个密文块异或
        xor_block = ''.join([str(int(a) ^ int(b)) for a, b in zip(block, previous_cipherblock)])

        cipherblock = saes.encrypt(xor_block)


      
        ciphertext += cipherblock
        blocks.append(cipherblock)
        previous_cipherblock = cipherblock
    if tamper_first_block:
        first_block = blocks[0]
        tampered_block = first_block[:15] + ('1' if first_block[15] == '0' else '0')
        ciphertext = ciphertext.replace(first_block, tampered_block, 1)
    # 将结果转换为十六进制字符串
    hex_ciphertext = hex(int(ciphertext, 2))[2:].zfill(len(ciphertext) // 4)

    return hex_ciphertext


In [145]:
ciphertext1 = cbc_encrypt_tamper(plaintext,key,iv)
plaintext1 = cbc_decrypt(ciphertext1,key,iv)

ciphertext2 = cbc_encrypt_tamper(plaintext,key,iv,True)
plaintext2 = cbc_decrypt(ciphertext2,key,iv)

print("未被篡改的密文:",ciphertext1)
print("被篡改后的密文:",ciphertext2)
# 
print("未被篡改的解密明文:",plaintext1)
print("被篡改后的解密明文:",plaintext2)

未被篡改的密文: a95ea41bda4db7b2672fc73a5af31b90e6444fc43b63
被篡改后的密文: a95fa41bda4db7b2672fc73a5af31b90e6444fc43b63
未被篡改的解密明文: Hello S-AES and CBC!
被篡改后的解密明文: (klmo S-AES and CBC!


可以看到，密文在已经形成之后被篡改会再带来后一个block的更改，其余不变。

这种篡改在没有其余安全措施的情况下依然不安全，篡改攻击精度高。

### 5.3 针对CBC的攻击
#### **1. 字节反转攻击:**
如果我们改变Ciphertext-N-1中的一个字节，然后和下一块解密后的密文xor，就可以得到一个不同的明文，而这个明文是我们可以控制的。

**举个例子**
> 我们想要更改某密文块C1，其对应明文Hello,更改为Jello

**第一步：计算差异**
    Hello XOR Jello = Δ

**第二步：将差异应用于C1**
    C1' = C1 XOR Δ
    
**能修改成功的原因是：**
    D(C1') XOR IV = D(C1 XOR Δ) XOR IV = (D(C1) XOR IV) XOR Δ = Hello XOR Δ = Jello
    C1‘会被先解密，于是经过逆运算得到我们想要篡改的'Jello'

并且，由于CBC解密过程中单个密文块只有本身和之后一个block，使得篡改变得容易。

### 5.4 结果分析

经过**5.2**的实例和**5.3**的分析,我们得到结论：

没有其他安全措施的CBC工作模式容易受到篡改攻击和字节替换攻击。